In [ ]:
# Upgrade Oracle ADS to pick up latest features and maintain compatibility with Oracle Cloud Infrastructure.

!pip install -U oracle-ads

Oracle Data Science service sample notebook.

Copyright (c) 2022 Oracle, Inc. All rights reserved. Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.

***

# <font color=red>Caltech Pedestrian Detection Benchmark Repository</font>
<p style="margin-left:10%; margin-right:10%;">by <font color="teal">Oracle for Research</font></p>

---

## Overview:

The Caltech Pedestrian Detection Benchmark dataset consists of approximately 10 hours of 640x480 30Hz video taken from a vehicle driving through regular traffic in an urban environment. The data consists of about 250,000 frames in 137 approximately one-minute-long segments. There are approximately 350,000 bounding boxes and 2,300 unique pedestrians have been annotated. The annotation includes temporal correspondence between bounding boxes and detailed occlusion labels.

This notebook demonstrates how to download the data from Oracle Cloud Infrastructure (OCI) Object Storage. It helps you understand the data and extract images from `.seq` files to a target folder.

Developed on [General Machine Learning](https://docs.oracle.com/iaas/data-science/using/conda-gml-fam.htm) for CPU on Python 3.7 (version 1.0)

---

## Contents:

- <a href="#intro">Introduction</a>
    - <a href='#data'>Dataset</a>
    - <a href="#open_data">What is Oracle Open Data?</a>
- <a href='#download'>Downloading Dataset</a>
- <a href='#process'>Processing Raw Data</a>
    - <a href='#metadata'>Extract Metadata</a>
    - <a href='#image'>Extract Image Data</a>
- <a href="#cleanup">Clean Up</a>
- <a href='#ref'>References</a>

---

Datasets are provided as a convenience. Datasets are considered third-party content and are not considered materials under your agreement with Oracle.
    
You can access the [Caltech Pedestrian Detection Benchmark](http://www.vision.caltech.edu/archive.html) dataset license [here](https://creativecommons.org/licenses/by/4.0/).

---


In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import requests
import struct
import tarfile

from os.path import join
from shutil import rmtree
from tempfile import mkdtemp

<a id="intro"></a>
# Introduction

<a id='data'></a>
## Dataset

**Data**: The dataset consists of training and testing data. The training data are in the set00 to set05 directories. The testing data are in the set06 to set10 directories. Both of which are about 1 GB and contain 6 to 13 one-minute-long `.seq` files along with annotation. Refer to the papers [Pedestrian Detection: An Evaluation of the State of the Art](http://www.vision.caltech.edu/Image_Datasets/CaltechPedestrians/files/PAMI12pedestrians.pdf) and [Pedestrian Detection: A Benchmark](http://www.vision.caltech.edu/Image_Datasets/CaltechPedestrians/files/CVPR09pedestrians.pdf) for more details.

**Download Template**: `https://objectstorage.us-ashburn-1.oraclecloud.com/n/idcxvbiyd8fn/b/caltech/o/<tar file or res directory>/<zip file>`. For example, 
* https://objectstorage.us-ashburn-1.oraclecloud.com/n/idcxvbiyd8fn/b/caltech/o/set00.tar
* https://objectstorage.us-ashburn-1.oraclecloud.com/n/idcxvbiyd8fn/b/caltech/o/res/AFS.zip

**Directory Structure**: The training data and testing data are available in `.tar` files along with the annotation in `annotations.zip`. Output files containing detection results for all evaluated algorithms are available in the `res` directory.

**Data Availability**: All data is available from the [Caltech Pedestrian Detection Benchmark repository](https://opendata.oraclecloud.com/ords/r/opendata/opendata/details?data_set_id=22), which is part of [Oracle Open Data](https://opendata.oraclecloud.com/ords/r/opendata/opendata/home).

<a id="open_data"></a>
## What is Oracle Open Data?

Oracle Open Data is a free service that curates spatial images, protein sequences, and annotated text files from the world's leading scientific databases. The repository connects researchers, developers, students, and educators with petabytes of open data from trusted resources. You can use Oracle Open Data to view important metadata and sample code for each data set, which simplifies technical complexities and makes it easy for researchers to use.

<a id='download'></a>
# Downloading the Dataset

OCI Object Storage enables you to securely store any type of data in its native format. With built-in redundancy, Object Storage is ideal for building modern applications that require scale and flexibility because you can use it to consolidate multiple data sources for analytics, backup, or archive purposes.

The Caltech pedestrian detection benchmark for video data are stored in Object Storage in the SEQ format. You can run the next cell to download and extract the files to a directory. Use training set `set00` as an example.

In [ ]:
dataset = "set00"
seqs_url = f"https://objectstorage.us-ashburn-1.oraclecloud.com/n/idcxvbiyd8fn/b/caltech/o/{dataset}.tar"
print("Downloading training data {}...".format(dataset))
response = requests.get(seqs_url, stream=True)
file = tarfile.open(fileobj=response.raw, mode="r|tar")
data_path = mkdtemp()
print(f"Extracting data to {data_path}")
file.extractall(path=data_path)
print(f"Download complete in directory {join(data_path, dataset)}")

<a id='process'></a>
# Processing Raw Data

<a id='metadata'></a>
## Extract Metadata

Each `.seq` file is a series of concatenated image frames with a fixed-size header. Effectively, the format concatenates a collection of image files into a single file with predictable locations for each image within the file.

The header in a `.seq` file contains metadata like `width` and `height` of the images, `imageBitDepth` for the color information, `imageSizeInBytes`, `numFrames` that represents the total number of frames in the file, `fps` that represents the frequency at which consecutive frames are captured. In particular, each `imageFormat` is mapped to a specific code.

In [ ]:
metadata = [
    "width",
    "height",
    "imageBitDepth",
    "imageBitDepthReal",
    "imageSizeInBytes",
    "imageFormatCode",
    "numFrames",
]

image_format = {
    1: ".png",
    2: ".png",
    100: ".raw",
    101: ".brgb8",
    102: ".jpg",
    103: ".jbrgb",
    200: ".raw",
    201: ".jpg",
}

The function `get_header` is defined in the next cell. This function will read header data for a given sequence file. It hoes this by skipping the file header, which is the first 548 bytes. It then parses the header. For details on how this is done, refer to the [official Matlab code](http://www.vision.caltech.edu/Image_Datasets/CaltechPedestrians/code/code3.2.1.zip) for details about this position. 

In [ ]:
example_seq = join(data_path, dataset, "V000.seq")
HEADER_DATA_INDEX = 548


def get_header(seq):
    with open(seq, "rb") as seq_file:
        header = {}
        seq_file.seek(HEADER_DATA_INDEX)  # skip to actual data
        for info in metadata:
            header[info] = struct.unpack("I", seq_file.read(4))[
                0
            ]  # take bytes and convert to integer
        seq_file.read(4)  # skip next 4 bytes
        header["trueImageSize"] = struct.unpack("I", seq_file.read(4))[0]
        header["fps"] = struct.unpack("d", seq_file.read(8))[0]
        return header


example_header = get_header(example_seq)
example_header

<a id='image'></a>
## Extract Image Data

Before you extract the image data you will need a directory to store the images. The following cell does this.

In [ ]:
image_path = join(data_path, dataset, "images")
if not os.path.exists(image_path):
    os.makedirs(image_path)
    print(f"{image_path} created to store images")
else:
    print(f"{image_path} already exists")

The image data starts at the $1024^{th}$ byte in the `.seq` file. The function `output_images` reads the image data and saves images to the `image_path` directory. The `seq` parameter is the original data `.seq` file you download, and `header` is the returned map from `get_header` function.

In [ ]:
IMAGE_DATA_INDEX = 1024


def output_images(seq, header):
    with open(seq, "rb") as seq_file:
        seq_file.seek(IMAGE_DATA_INDEX)  # skip to actual image data
        for img_id in range(header["numFrames"]):
            img_size = struct.unpack("I", seq_file.read(4))[0]
            img_data = seq_file.read(img_size)
            # map to the image format from the corresponding code
            img_name = str(img_id) + image_format[header["imageFormatCode"]]
            # write image data to a specific path
            path = os.path.join(image_path, img_name)
            with open(path, "wb") as file:
                file.write(bytearray(img_data))

            seq_file.seek(12, 1)  # skip to next image
        print(f"{header['numFrames']} images were extracted to {image_path}")


output_images(example_seq, example_header)

Plot one of the images.

In [ ]:
plt.imshow(mpimg.imread(join(image_path, "0.jpg")))
plt.axis("off")
plt.show()

<a id="cleanup"></a>
# Clean Up

This notebook downloaded over a gigabyte of data. This section will delete those files.

In [ ]:
rmtree(data_path)

<a id='ref'></a>
# References

- [ADS Library Documentation](https://accelerated-data-science.readthedocs.io/en/latest/index.html)
- [Data Science YouTube Videos](https://www.youtube.com/playlist?list=PLKCk3OyNwIzv6CWMhvqSB_8MLJIZdO80L)
- [OCI Data Science Documentation](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm)
- [Oracle Data & AI Blog](https://blogs.oracle.com/datascience/)
- [Understanding Conda Environments](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/use-notebook-sessions.htm#conda_understand_environments)
- [Use Resource Manager to Configure Your Tenancy for Data Science](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/orm-configure-tenancy.htm)
- [Pedestrian Detection: An Evaluation of the State of the Art](http://www.vision.caltech.edu/Image_Datasets/CaltechPedestrians/files/PAMI12pedestrians.pdf)
- [Pedestrian Detection: A Benchmark](http://www.vision.caltech.edu/Image_Datasets/CaltechPedestrians/files/CVPR09pedestrians.pdf)
- [Description of SeqIo](https://pdollar.github.io/toolbox/videos/seqIo.html)
- [Caltech Pedestrian Detection Benchmark Python Extractor](https://github.com/martinkersner/caltech-pedestrian-detection-benchmark-python-extractor)